In [56]:
import torch
import optuna
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split, KFold
import os
import polars as pl

import yaml
with open('config.yaml','r') as file:
    config=yaml.safe_load(file)
dataset_type="1d_raw_std"
target_type="gas"
x_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_train_{dataset_type}.npy")
t_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_train_{dataset_type}.npy")
x_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
t_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_test_{dataset_type}.npy")

bdir="/mnt/sdb/yyamaguchi/simulationB4/dataset"
odir=os.path.join(bdir,f"outputs_issei_{dataset_type}_{target_type}")
os.makedirs(odir,exist_ok=True)
device="cuda:0"
W     = config['hyperparameter']['input_length']
H     = config['hyperparameter']['input_height']
C     = config['hyperparameter']['input_channel']
dropout = config['hyperparameter']['dropout']
ksize1 = config['hyperparameter']['ksize1']
ksize2 = config['hyperparameter']['ksize2']
stride1 = config['hyperparameter']['stride1']
stride2 = config['hyperparameter']['stride2']
maxpool = config['hyperparameter']['maxpool']
SEED = config['hyperparameter']['seed']
kfolds = config['hyperparameter']['kfolds']

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    # CUDAがある場合
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

class SimpleCNNissei(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNNissei, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(4*input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.5)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel*4).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,4*self.mid_channel*i:4*self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

# class SimpleCNN(nn.Module):
#     def __init__(self, input_length, input_channel, mid_channel=16):
#         super(SimpleCNN, self).__init__()
#         self.input_channel = input_channel
#         self.input_length = input_length
#         self.mid_channel = mid_channel
#         self.models = nn.ModuleList([
#             self._build_sub_network(mid_channel) for _ in range(input_channel)
#         ])
#         self.fc = nn.Linear(input_channel*mid_channel,2)
#         # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
#         # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
#     def _build_sub_network(self, mid_channel):
#         return nn.Sequential(
#             nn.Conv1d(1, mid_channel, kernel_size=ksize1, stride=stride1),
#             nn.BatchNorm1d(mid_channel),
#             nn.ReLU(),
#             nn.MaxPool1d(maxpool),
#             nn.Conv1d(mid_channel, mid_channel, kernel_size=ksize2, stride=stride2),
#             nn.BatchNorm1d(mid_channel),
#             nn.ReLU(),
#             nn.MaxPool1d(maxpool),
#             nn.AdaptiveAvgPool1d(1),
#             nn.Dropout(dropout)
#         )

#     def forward(self, xall):
#         bsize = xall.shape[0]
#         output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
#         for i in range(self.input_channel):
#             x = xall[:,i]
#             x = x.unsqueeze(1)
#             x = self.models[i](x)
#             x = x.view(x.size(0), -1)
#             output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
#         output = self.fc(output)
#         return output.squeeze(1)

print(x_train.shape)
print(t_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()

def objective(trial):
    curodir=os.path.join(odir,f"case{trial.number}")
    wdir=os.path.join(curodir,"weights")
    ldir=os.path.join(curodir,"logs")
    os.makedirs(curodir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    lr    = trial.suggest_float("lr",1e-6,1e-2,log=True)
    N     = trial.suggest_int("num_epochs",30,100)
    B     = trial.suggest_categorical("batch_size",[16,32,64])
    l2    = trial.suggest_float("l2_lambda",1e-9,1e-3,log=True)
    res_list=[]
    coefs_list=[]
    reg_list=[]
    coefg_list=[]
    set_seed(SEED)
    kf = KFold(n_splits=kfolds, shuffle=True, random_state=SEED)
    dataset=TensorDataset(x,t)
    Nttl=len(dataset)
    rng=np.random.default_rng()
    xt=torch.from_numpy(x_test).float()
    tt=torch.from_numpy(t_test).float()
    testset=TensorDataset(xt,tt)
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    testsize=len(testset)
    best_val_loss_list=[]

    for k, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        trasize=len(train_idx)
        valsize=len(val_idx)
        testresult=np.zeros((4,valsize))
        # trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
        trainset=Subset(dataset, train_idx)
        valset = Subset(dataset, val_idx)
        trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
        valloader=DataLoader(valset,batch_size=1,shuffle=False)
        # testloader=DataLoader(testset,batch_size=1,shuffle=False)
        model=SimpleCNNissei(W,C).to(device)

        # criterion=nn.HuberLoss(delta=delta)
        criterion=nn.MSELoss()
        optimizer=optim.Adam(params=model.parameters(),lr=lr,weight_decay=l2)
        tlosshistory=[]
        vlosshistory=[]

        for epoch in range(N):
            model.train()
            rloss=0.0
            for bx, by in trainloader:
                # start_idx=rng.integers(250)
                start_idx=0
                bx=bx[:,:,start_idx:start_idx+W]
                bx=bx.to(device)
                by=by.to(device)
                optimizer.zero_grad()
                o=model(bx)
                loss=criterion(o,by)
                loss.backward()
                optimizer.step()
                rloss=rloss+loss.item()*bx.size(0)
            eloss=rloss/len(trainloader.dataset)
            tlosshistory.append(eloss)

            model.eval()
            vrloss=0.0
            with torch.no_grad():
                for vx,vy in valloader:
                    vx=vx[:,:,:W]
                    vx=vx.to(device)
                    vy=vy.to(device)
                    vo=model(vx)
                    vloss=criterion(vo,vy)
                    vrloss+=vloss.item()*vx.size(0)
            veloss=vrloss/len(valloader.dataset)
            vlosshistory.append(veloss)

        torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
        plt.figure()
        plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
        plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
        plt.title('Learning Curve (Log Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Log(Loss)')
        plt.rcParams["font.size"] = 16
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
        plt.close()

        plt.figure()
        plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
        plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
        plt.title('Learning Curve (Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.rcParams["font.size"] = 20
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
        plt.close()

        model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
        model=model.to(device)
        model.eval()
        pred=[]
        targ=[]
        with torch.no_grad():
            for tx,ty in valloader:
                tx=tx[:,:,:W]
                tx=tx.to(device)
                ty=ty.to(device)
                o=model(tx)
                pred.append(o.cpu())
                targ.append(ty.cpu())
        pred=torch.cat(pred,dim=0)
        targ=torch.cat(targ,dim=0)

        pred=pred.numpy().transpose()
        targ=targ.numpy().transpose()
        targ[0,:]=targ[0,:]/10
        pred[0,:]=pred[0,:]/10

        testresult[0,:]=targ[0,:]
        testresult[1,:]=pred[0,:]
        testresult[2,:]=targ[1,:]
        testresult[3,:]=pred[1,:]
        np.save(os.path.join(wdir,f'result{k}.npy'),testresult)

        corr=np.corrcoef(targ[0,:],pred[0,:])
        coef=corr[0,1]
        print(f'coefficient solid: {coef}')
        re=np.sqrt(np.sum(np.power(targ[0,:]-pred[0,:],2))/targ.shape[1])
        print(f're solid: {re}')
        res_list.append(re)
        coefs_list.append(coef)
        corr=np.corrcoef(targ[1,:],pred[1,:])
        coef=corr[0,1]
        print(f'coefficient void: {coef}')
        re=np.sqrt(np.sum(np.power(targ[1,:]-pred[1,:],2))/targ.shape[1])
        print(f're void: {re}')
        reg_list.append(re)
        coefg_list.append(coef)

        plt.figure(figsize=(8, 7))
        plt.scatter(targ[1,:], pred[1,:], alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.4)
        plt.ylim(0, 0.4)
        plt.xticks([0,0.1,0.2,0.3,0.4])
        plt.yticks([0,0.1,0.2,0.3,0.4])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
        plt.close()
        plt.figure(figsize=(8, 7))
        plt.scatter(targ[0,:], pred[0,:], alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.06)
        plt.ylim(0, 0.06)
        plt.xticks([0,0.02,0.04,0.06])
        plt.yticks([0,0.02,0.04,0.06])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
        plt.close()

    reg_list=np.array(reg_list)
    coefg_list=np.array(coefg_list)
    res_list=np.array(res_list)
    coefs_list=np.array(coefs_list)
    print(f"res:{np.mean(res_list)}")
    print(f"coefs:{np.mean(coefs_list)}")
    print(f"reg:{np.mean(reg_list)}")
    print(f"coefg:{np.mean(coefg_list)}")
    df = pl.DataFrame({
        "RMSE":reg_list,
        "R2":coefg_list
    })
    mean = df.mean()
    std = df.std()

    df = pl.concat([
        df,
        mean,
        std
    ])
    df.write_csv(os.path.join(curodir,"result_void.csv"))
    df = pl.DataFrame({
        "RMSE":res_list,
        "R2":coefs_list
    })
    mean = df.mean()
    std = df.std()

    df = pl.concat([
        df,
        mean,
        std
    ])
    df.write_csv(os.path.join(curodir,"result_solid.csv"))
    trial.set_user_attr("r2scores", np.mean(coefs_list))
    trial.set_user_attr("r2scoreg", np.mean(coefg_list))
    trial.set_user_attr("rmseg", np.mean(reg_list))
    trial.set_user_attr("rmses", np.mean(res_list))

    config["best_hyperparameters"]={
        "lr":float(lr),
        "num_epochs": int(N),
        "batch_size": int(B),
        "l2_lambda": float(l2),
        "metrics": {
            "rmse_void": float(np.mean(reg_list)),
            "r2_void":float(np.mean(coefg_list)),
            "rmse_solid": float(np.mean(res_list)),
            "r2_solid":float(np.mean(coefs_list))
        }
    }
    with open(os.path.join(curodir,"config.yaml"),"w") as f:
        yaml.dump(config,f,default_flow_style=False,sort_keys=False)

    return np.mean(reg_list)+np.mean(res_list)*10

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

best_trial=study.best_trial
best_rmse=best_trial.value 
best_params=best_trial.params
best_r2_solid = best_trial.user_attrs["r2scores"]
best_r2_void = best_trial.user_attrs["r2scoreg"]
best_rmse_solid = best_trial.user_attrs["rmses"]
best_rmse_void = best_trial.user_attrs["rmseg"]

print(f"Best trial number: {study.best_trial.number}")

config["best_hyperparameters"]={
    "lr":float(best_params["lr"]),
    "num_epochs": int(best_params["num_epochs"]),
    "batch_size": int(best_params["batch_size"]),
    "l2_lambda": float(best_params["l2_lambda"]),
    "metrics": {
            "rmse_void": float(best_rmse_void),
            "r2_void":float(best_r2_void),
            "rmse_solid": float(best_rmse_solid),
            "r2_solid":float(best_r2_solid)
    }
}
with open(os.path.join(odir,"config.yaml"),"w") as f:
    yaml.dump(config,f,default_flow_style=False,sort_keys=False)

[I 2026-01-14 21:51:13,547] A new study created in memory with name: no-name-acaf5f9a-cbbf-4c5c-87c0-841e6958b578


(483, 2, 3000)
(483,)


/tmp/ipykernel_3747402/1912012082.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
[W 2026-01-14 2

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
import torch
import optuna
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split, KFold
import os
import polars as pl

import yaml
with open('config.yaml','r') as file:
    config=yaml.safe_load(file)
dataset_type="1d_raw_std"
target_type="gas"
x_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_train_{dataset_type}.npy")
t_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_train_{dataset_type}.npy")
x_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
t_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_test_{dataset_type}.npy")

bdir="/mnt/sdb/yyamaguchi/simulationB4/dataset"
odir=os.path.join(bdir,f"outputs_issei2_{dataset_type}_{target_type}")
os.makedirs(odir,exist_ok=True)
device="cuda:0"
W     = config['hyperparameter']['input_length']
H     = config['hyperparameter']['input_height']
C     = config['hyperparameter']['input_channel']
dropout = config['hyperparameter']['dropout']
ksize1 = config['hyperparameter']['ksize1']
ksize2 = config['hyperparameter']['ksize2']
stride1 = config['hyperparameter']['stride1']
stride2 = config['hyperparameter']['stride2']
maxpool = config['hyperparameter']['maxpool']
SEED = config['hyperparameter']['seed']
kfolds = config['hyperparameter']['kfolds']

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    # CUDAがある場合
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

class SimpleCNNissei(nn.Module):
    def __init__(self, input_length, input_channel, p, mid_channel=32):
        super(SimpleCNNissei, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.p = p
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(4*input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1,32, kernel_size=3, stride=1),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1),
            # nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1),
            # nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(self.p)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel*4).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,4*self.mid_channel*i:4*self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=ksize1, stride=stride1),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(maxpool),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=ksize2, stride=stride2),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(maxpool),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(dropout)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

print(x_train.shape)
print(t_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()

def objective(trial):
    curodir=os.path.join(odir,f"case{trial.number}")
    wdir=os.path.join(curodir,"weights")
    ldir=os.path.join(curodir,"logs")
    os.makedirs(curodir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    lr    = trial.suggest_categorical("lr",[1e-2,1e-3,1e-4,1e-5])
    N     = trial.suggest_int("num_epochs",50,250,step=100)
    B     = trial.suggest_categorical("batch_size",[16,32,64])
    l2    = trial.suggest_categorical("l2_lambda",[0.5,0.3,0.1])
    earlystopmax=15
    res_list=[]
    coefs_list=[]
    reg_list=[]
    coefg_list=[]
    set_seed(SEED)
    kf = KFold(n_splits=kfolds, shuffle=True, random_state=SEED)
    dataset=TensorDataset(x,t)
    Nttl=len(dataset)
    rng=np.random.default_rng()
    xt=torch.from_numpy(x_test).float()
    tt=torch.from_numpy(t_test).float()
    testset=TensorDataset(xt,tt)
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    testsize=len(testset)
    best_val_loss_list=[]

    for k, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        trasize=len(train_idx)
        valsize=len(val_idx)
        testresult=np.zeros((2,valsize))
        # trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
        trainset=Subset(dataset, train_idx)
        valset = Subset(dataset, val_idx)
        trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
        valloader=DataLoader(valset,batch_size=1,shuffle=False)
        # testloader=DataLoader(testset,batch_size=1,shuffle=False)
        model=SimpleCNNissei(W,C,l2).to(device)

        # criterion=nn.HuberLoss(delta=delta)
        criterion=nn.MSELoss()
        optimizer=optim.Adam(params=model.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
        tlosshistory=[]
        vlosshistory=[]
        veloss_best=10000
        earlystopcount=0
        epochttl=0

        for epoch in range(N):
            model.train()
            rloss=0.0
            for bx, by in trainloader:
                # start_idx=rng.integers(250)
                start_idx=0
                bx=bx[:,:,start_idx:start_idx+W]
                bx=bx.to(device)
                by=by.to(device)
                optimizer.zero_grad()
                o=model(bx)
                loss=criterion(o,by)
                loss.backward()
                optimizer.step()
                rloss=rloss+loss.item()*bx.size(0)
            eloss=rloss/len(trainloader.dataset)
            tlosshistory.append(eloss)

            model.eval()
            vrloss=0.0
            with torch.no_grad():
                for vx,vy in valloader:
                    vx=vx[:,:,:W]
                    vx=vx.to(device)
                    vy=vy.to(device)
                    vo=model(vx)
                    vloss=criterion(vo,vy)
                    vrloss+=vloss.item()*vx.size(0)
            veloss=vrloss/len(valloader.dataset)
            scheduler.step(veloss)
            vlosshistory.append(veloss)
            epochttl+=1
            if veloss<veloss_best:
                earlystopcount=0
                veloss_best=veloss
                torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
            else:
                earlystopcount+=1
            if earlystopcount==earlystopmax:
                print(f"Epoch ended at {epoch}")
                break

        plt.figure()
        plt.plot(range(1, epochttl + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
        plt.plot(range(1, epochttl + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
        plt.title('Learning Curve (Log Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Log(Loss)')
        plt.rcParams["font.size"] = 16
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
        plt.close()

        plt.figure()
        plt.plot(range(1, epochttl + 1), tlosshistory, label='Train Loss')
        plt.plot(range(1, epochttl + 1), vlosshistory, label='Validation Loss')
        plt.title('Learning Curve (Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.rcParams["font.size"] = 20
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
        plt.close()

        model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
        model=model.to(device)
        model.eval()
        pred=[]
        targ=[]
        with torch.no_grad():
            for tx,ty in valloader:
                tx=tx[:,:,:W]
                tx=tx.to(device)
                ty=ty.to(device)
                o=model(tx)
                pred.append(o.cpu())
                targ.append(ty.cpu())
        pred=torch.cat(pred,dim=0)
        targ=torch.cat(targ,dim=0)

        pred=pred.numpy()
        targ=targ.numpy()

        testresult[0,:]=targ
        testresult[1,:]=pred
        np.save(os.path.join(wdir,f'result{k}.npy'),testresult)

        corr=np.corrcoef(targ,pred)
        coef=corr[0,1]
        print(f'coefficient void: {coef}')
        re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
        print(f're void: {re}')
        reg_list.append(re)
        coefg_list.append(coef)

        if target_type=="gas":
            plt.figure(figsize=(8, 7))
            plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
            plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
            plt.title('Predicted vs Actual Values')
            plt.xlabel('Actual Value')
            plt.ylabel('Predicted Value')
            plt.xlim(0, 0.4)
            plt.ylim(0, 0.4)
            plt.xticks([0,0.1,0.2,0.3,0.4])
            plt.yticks([0,0.1,0.2,0.3,0.4])
            plt.rcParams["font.size"] = 20
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
            plt.close()
        if target_type=="solid":
            plt.figure(figsize=(8, 7))
            plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
            plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
            plt.title('Predicted vs Actual Values')
            plt.xlabel('Actual Value')
            plt.ylabel('Predicted Value')
            plt.xlim(0, 0.06)
            plt.ylim(0, 0.06)
            plt.xticks([0,0.02,0.04,0.06])
            plt.yticks([0,0.02,0.04,0.06])
            plt.rcParams["font.size"] = 20
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
            plt.close()

    reg_list=np.array(reg_list)
    coefg_list=np.array(coefg_list)
    print(f"reg:{np.mean(reg_list)}")
    print(f"coefg:{np.mean(coefg_list)}")
    df = pl.DataFrame({
        "RMSE":reg_list,
        "R2":coefg_list
    })
    mean = df.mean()
    std = df.std()

    df = pl.concat([
        df,
        mean,
        std
    ])
    df.write_csv(os.path.join(curodir,"result.csv"))
    trial.set_user_attr("r2score", np.mean(coefg_list))
    trial.set_user_attr("rmse", np.mean(reg_list))

    config["best_hyperparameters"]={
        "lr":float(lr),
        "num_epochs": int(N),
        "batch_size": int(B),
        "l2_lambda": float(l2),
        "metrics": {
            "rmse": float(np.mean(reg_list)),
            "r2":float(np.mean(coefg_list))
        }
    }
    with open(os.path.join(curodir,"config.yaml"),"w") as f:
        yaml.dump(config,f,default_flow_style=False,sort_keys=False)

    return np.mean(reg_list)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

best_trial=study.best_trial
best_rmse=best_trial.value 
best_params=best_trial.params
best_r2 = best_trial.user_attrs["r2score"]

config["best_hyperparameters"]={
    "lr":float(best_params["lr"]),
    "num_epochs": int(best_params["num_epochs"]),
    "batch_size": int(best_params["batch_size"]),
    "l2_lambda": float(best_params["l2_lambda"]),
    "metrics": {
            "rmse": float(best_rmse),
            "r2":float(best_r2)
    }
}
with open(os.path.join(odir,"config.yaml"),"w") as f:
    yaml.dump(config,f,default_flow_style=False,sort_keys=False)

print(f"Best trial number: {study.best_trial.number}")

/home/user01/Document/yyamaguchi/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2026-01-18 16:19:09,238] A new study created in memory with name: no-name-c2c76e66-0595-4a3a-b301-e8ff36f5595d


(483, 2, 3000)
(483,)


/tmp/ipykernel_3949160/4199930222.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


coefficient void: 0.8130714932196899
re void: 0.055977405646553124
coefficient void: 0.8113689876808546
re void: 0.05181398813969419
coefficient void: 0.8634930142363472
re void: 0.04991634599657196
coefficient void: 0.8467175263430838
re void: 0.0504770198119962


[I 2026-01-18 16:20:26,754] Trial 0 finished with value: 0.05214832344403024 and parameters: {'lr': 0.0001, 'num_epochs': 50, 'batch_size': 32, 'l2_lambda': 0.5}. Best is trial 0 with value: 0.05214832344403024.


...

coefficient void: 0.8334279766582263
re void: 0.05101689050379308
reg:0.04963169993488552
coefg:0.8493609492964668
Epoch ended at 170
coefficient void: 0.8293288748485119
re void: 0.05370872371440862
Epoch ended at 94
coefficient void: 0.8364962725843298
re void: 0.048809584410860335
Epoch ended at 83
coefficient void: 0.8884969047018068
re void: 0.045122610319481324
Epoch ended at 105
coefficient void: 0.8837521738206012
re void: 0.04428568174955477
Epoch ended at 78


[I 2026-01-18 17:08:23,321] Trial 18 finished with value: 0.04835986832971669 and parameters: {'lr': 0.01, 'num_epochs': 230, 'batch_size': 16, 'l2_lambda': 0.3}. Best is trial 18 with value: 0.04835986832971669.


coefficient void: 0.8421914814268882
re void: 0.04987274145427838
reg:0.04835986832971669
coefg:0.8560531414764275
Epoch ended at 170
coefficient void: 0.8293288748485119
re void: 0.05370872371440862
Epoch ended at 94
coefficient void: 0.8364962725843298
re void: 0.048809584410860335


[W 2026-01-18 17:10:23,211] Trial 19 failed with parameters: {'lr': 0.01, 'num_epochs': 230, 'batch_size': 16, 'l2_lambda': 0.3} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/user01/Document/yyamaguchi/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3949160/4199930222.py", line 197, in objective
    rloss=rloss+loss.item()*bx.size(0)
KeyboardInterrupt
[W 2026-01-18 17:10:23,212] Trial 19 failed with value None.


KeyboardInterrupt: 

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os
import polars as pl

dataset_type="1d_raw_std"
target_name="relative_ratiowww"
target_type="gas"
x_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_train_{dataset_type}.npy")
t_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_train_{dataset_type}.npy")
x_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
t_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_test_{dataset_type}.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/dataset"
odir=os.path.join(bdir,f"outputs_issei2_{dataset_type}_{target_type}")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=16
W=2500
H=49
lr=0.0001
N=150
l1=1e-7
# l2=5e-5
l2=0.3
delta=0.01
C=2
SEED=93
p=0.3
earlystopmax=15

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=51),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=51),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

class SimpleCNNissei(nn.Module):
    def __init__(self, input_length, input_channel, p, mid_channel=32):
        super(SimpleCNNissei, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.p = p
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(4*input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1),
            # nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1),
            # nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(self.p)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel*4).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,4*self.mid_channel*i:4*self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

class SimpleCNN2d(nn.Module):
    def __init__(self, input_width, input_height, input_channel, mid_channel=16):
        super(SimpleCNN2d, self).__init__()
        self.input_channel = input_channel
        self.input_width = input_width
        self.input_height = input_height
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv2d(1, mid_channel, kernel_size=16),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(mid_channel, mid_channel, kernel_size=4),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    # CUDAがある場合
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
xt=torch.from_numpy(x_test).float()
tt=torch.from_numpy(t_test).float()
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
dataset=TensorDataset(x,t)
testset=TensorDataset(xt,tt)
Nttl=len(dataset)
trasize=int(0.8*Nttl)
valsize=Nttl-trasize
# testsize=Nttl-trasize-valsize
set_seed(SEED)

for k in range(10):
    cur_seed=SEED+k
    gen_split=torch.Generator()
    gen_split.manual_seed(cur_seed)
    testresult=np.zeros((2,valsize))
    trainset,valset=random_split(dataset,[trasize,valsize],generator=gen_split)
    valloader=DataLoader(valset,batch_size=1,shuffle=False)
    model=SimpleCNNissei(W,C,p).to(device)
    # params = 0
    # for p in model.parameters():
    #     if p.requires_grad:
    #         params += p.numel()
            
    # print(params)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
    tlosshistory=[]
    vlosshistory=[]
    loaded_target=[]
    veloss_best=10000
    earlystopcount=0
    epochttl=0
    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)

    for epoch in range(N):
        cur_seed=SEED*2+200*k+epoch
        gen_loader=torch.Generator()
        gen_loader.manual_seed(cur_seed)
        trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            first_by=by[0].item()
            loaded_target.append(first_by)
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            # l1n=sum(p.abs().sum() for p in model.parameters())
            # l2n=sum(p.pow(2).sum() for p in model.parameters())
            # loss=loss+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        scheduler.step(veloss)
        vlosshistory.append(veloss)
        epochttl+=1
        if veloss<veloss_best:
            earlystopcount=0
            veloss_best=veloss
            torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
        else:
            earlystopcount+=1
        if earlystopcount==earlystopmax:
            print(f"Epoch ended at {epoch}")
            break

    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, epochttl + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, epochttl + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, epochttl + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, epochttl + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in valloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()
    loaded_target=np.array(loaded_target)

    testresult[0,:]=targ
    testresult[1,:]=pred
    df = pl.DataFrame({
        "target":targ,
        "prediction":pred
    })
    np.save(os.path.join(wdir,f'result{k}.npy'),testresult)
    df.write_csv(os.path.join(wdir,f"result_val{k}.csv"))
    df = pl.DataFrame({
        "target":loaded_target
    })
    df.write_csv(os.path.join(wdir,f"label_train{k}.csv"))

    corr=np.corrcoef(targ,pred)
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    if target_name=="relative_ratio" or target_name=="relative_ratio_lowgas" or target_name=="relative_ratio_highgas":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.xticks([0,0.25,0.5,0.75,1.0])
        plt.yticks([0,0.25,0.5,0.75,1.0])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()

    elif target_type == "gas":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.4)
        plt.ylim(0, 0.4)
        plt.xticks([0,0.1,0.2,0.3,0.4])
        plt.yticks([0,0.1,0.2,0.3,0.4])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()
    elif target_type == "solid":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.06)
        plt.ylim(0, 0.06)
        plt.xticks([0,0.02,0.04,0.06])
        plt.yticks([0,0.02,0.04,0.06])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()

reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")
df = pl.DataFrame({
    "RMSE":reg_list,
    "R2":coefg_list
})
mean = df.mean()
std = df.std()

df = pl.concat([
    df,
    mean,
    std
])
df.write_csv(os.path.join(odir,"result.csv"))

(483, 2, 3000)


/tmp/ipykernel_3949160/4001690143.py:284: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


coefficient void: 0.8276567154900984
re void: 0.05314686685477869
coefficient void: 0.8344264355632872
re void: 0.05080282976449638
coefficient void: 0.8112926327284167
re void: 0.05292191572631511
coefficient void: 0.8686830780616257
re void: 0.04674460111581632
coefficient void: 0.8426584797630101
re void: 0.055700303090595135
coefficient void: 0.8569342215703812
re void: 0.04978364554321218
Epoch ended at 33
coefficient void: 0.8364896475954718
re void: 0.056882948125049654
Epoch ended at 108
coefficient void: 0.8658265498340568
re void: 0.04892591456837374
Epoch ended at 134
coefficient void: 0.8897192967093844
re void: 0.04287995492162307
Epoch ended at 72
coefficient void: 0.890480726442055
re void: 0.0520558674809485
reg_list:[0.05314687 0.05080283 0.05292192 0.0467446  0.0557003  0.04978365
 0.05688295 0.04892591 0.04287995 0.05205587]
coefg_list:[0.82765672 0.83442644 0.81129263 0.86868308 0.84265848 0.85693422
 0.83648965 0.86582655 0.8897193  0.89048073]
reg:0.05098448471912

In [6]:
k=4

print(wdir)
model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
print(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
x_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
t_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_test_{dataset_type}.npy")
xt=torch.from_numpy(x_test).float().to(device)
tt=torch.from_numpy(t_test).float().to(device)
testset=TensorDataset(xt,tt)
testloader=DataLoader(testset,batch_size=1,shuffle=False)
testresult=np.zeros((2,len(testset)))
model=model.to(device)
model.eval()
pred=[]
targ=[]
with torch.no_grad():
    for tx,ty in testloader:
        tx=tx.to(device)
        ty=ty.to(device)
        o=model(tx)
        pred.append(o.cpu())
        targ.append(ty.cpu())
pred=torch.cat(pred,dim=0)
targ=torch.cat(targ,dim=0)

pred=pred.numpy()
targ=targ.numpy()

testresult[0,:]=targ
testresult[1,:]=pred
np.save(os.path.join(odir,f'result{k}.npy'),testresult)

corr=np.corrcoef(targ,pred)
coef=corr[0,1]
print(f'coefficient void: {coef}')
# re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
print(f're void: {re}')

if target_type == "gas":
    plt.figure(figsize=(8, 7))
    plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.xticks([0,0.1,0.2,0.3,0.4])
    plt.yticks([0,0.1,0.2,0.3,0.4])
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(odir, f'val_pred_vs_actual{k}.png'))
    plt.close()
if target_type == "solid":
    plt.figure(figsize=(8, 7))
    plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.06)
    plt.ylim(0, 0.06)
    plt.xticks([0,0.02,0.04,0.06])
    plt.yticks([0,0.02,0.04,0.06])
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(odir, f'val_pred_vs_actual{k}.png'))
    plt.close()

df = pl.DataFrame({
    "RMSE":re,
    "R2":coef
})
df.write_csv(os.path.join(odir,f"testresult{k}.csv"))

/mnt/sdb/yyamaguchi/simulationB4/dataset/outputs_issei_1d_bp_std_gas/weights
/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_1d_bp_std.npy


/tmp/ipykernel_3925435/1231364475.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


coefficient void: 0.8010008159444832
re void: 0.05703502079739214


In [30]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_two_env_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/tg_train_two_env_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_2env_std")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=32
W=2500
lr=0.001
N=50
l1=1e-7
l2=1e-6
C=2

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(input_length,mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, input_length, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

# class SimpleCNN(nn.Module):
#     def __init__(self, input_length, input_channel, mid_channel=16):
#         super(SimpleCNN, self).__init__()
#         self.input_channel = input_channel
#         self.input_length = input_length
#         self.mid_channel = mid_channel
#         self.conv1 = nn.Conv1d(1, mid_channel, kernel_size=201)
#         self.bn1 = nn.BatchNorm1d(mid_channel)
#         self.maxpool1 = nn.MaxPool1d(4,stride=1)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv1d(mid_channel, mid_channel, kernel_size=51)
#         self.bn2 = nn.BatchNorm1d(mid_channel)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool1d(4,stride=1)
#         self.pool = nn.AdaptiveAvgPool1d(1)
#         self.fc = nn.Linear(mid_channel*input_channel, 1)
#         self.dropout = nn.Dropout(0.4)
#         # self.ln1 = nn.LayerNorm([16, input_length])
#         # self.ln2 = nn.LayerNorm([4, input_length+2])
#     def forward(self, xall):
#         bsize = xall.shape[0]
#         output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
#         for i in range(self.input_channel):
#             x = xall[:,i,:]
#             x = x.unsqueeze(1)
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu1(x)
#             x = self.maxpool1(x)
#             # x = self.ln1(x)
#             x = self.conv2(x)
#             x = self.bn2(x)
#             x = self.relu2(x)
#             x = self.maxpool2(x)
#             # x = self.ln2(x)
#             x = self.pool(x)
#             x = self.dropout(x)
#             x = x.view(x.size(0), -1)
#             output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
#         output = self.fc(output)
#         return output.squeeze(1)

class GradCAM1d:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def __call__(self, input_tensor):
        self.model.eval()
        input_tensor = input_tensor.to(next(self.model.parameters()).device)
        self.model.zero_grad()
        out = self.model(input_tensor)
        print(out)
        if isinstance(out, (tuple, list)):
            out = out[0]
        target = out.squeeze()
        print(target)
        if target.ndim > 0:
            target = target.sum()
        self.model.zero_grad()
        target.backward(retain_graph=True)
        gradients = self.gradients         # [B, C, L]
        print(gradients.shape)
        activations = self.activations     # [B, C, L]
        print(activations.shape)
        weights = gradients.mean(dim=2, keepdim=True)  # [B, C, 1]
        grad_cam_map = (weights * activations).sum(dim=1, keepdim=True)  # (B,1,L)
        grad_cam_map = torch.relu(grad_cam_map)
        grad_cam_map = torch.nn.functional.interpolate(
            grad_cam_map, size=input_tensor.shape[2], mode='linear', align_corners=False
        )
        grad_cam_map = grad_cam_map.squeeze().cpu().numpy()
        grad_cam_map = (grad_cam_map - grad_cam_map.min()) / (grad_cam_map.max() - grad_cam_map.min() + 1e-8)
        return grad_cam_map

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

print(x_train.shape)
# x_train=x_train.reshape(x_train.shape[0],-1)
# x_train=x_train[:,[1,3,4],:]
print(t_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
# x=x.unsqueeze(1)
print(x.shape)
amp=10
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
xgradcam=x[50,:,:]
xgradcam=xgradcam.unsqueeze(0)
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize

for k in range(10):
    trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=1,shuffle=False)
    model=SimpleCNN(W,C).to(device)
    params = 0
    for p in model.parameters():
        if p.requires_grad:
            params += p.numel()
            
    print(params)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            # l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    print(pred.shape)
    print(targ.shape)

    # corr=np.corrcoef(targ[:,0],pred[:,0])
    # coef=corr[0,1]
    # print(f'coefficient solid: {coef}')
    # # re=np.mean(np.abs(targ[:,0]-pred[:,0])/(targ[:,0]+1e-7))
    # re=np.sqrt(np.sum(np.power(targ[:,0]-pred[:,0],2))/targ.shape[0])
    # print(f're solid: {re}')
    # res_list.append(re)
    # coefs_list.append(coef)
    # corr=np.corrcoef(targ[:,1],pred[:,1])
    # coef=corr[0,1]
    # print(f'coefficient void: {coef}')
    # # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    # re=np.sqrt(np.sum(np.power(targ[:,1]-pred[:,1],2))/targ.shape[0])
    # print(f're void: {re}')
    # reg_list.append(re)
    # coefg_list.append(coef)

    # plt.figure(figsize=(8, 8))
    # plt.scatter(targ[:,0], pred[:,0], alpha=0.6, marker='o', label='Solid fraction')
    # plt.plot([-1, 1], [-1, 1], 'r--', label='Ideal (y=x)')
    # plt.title('Predicted vs Actual Values')
    # plt.xlabel('Actual Value')
    # plt.ylabel('Predicted Value')
    # plt.xlim(-0.0, 0.05)
    # plt.ylim(-0.0, 0.05)
    # plt.rcParams["font.size"] = 20
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
    # plt.close()

    # plt.figure(figsize=(8, 8))
    # plt.scatter(targ[:,1], pred[:,1], alpha=0.6, marker='o', label='Void fraction')
    # plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    # plt.title('Predicted vs Actual Values')
    # plt.xlabel('Actual Value')
    # plt.ylabel('Predicted Value')
    # plt.xlim(0, 0.4)
    # plt.ylim(0, 0.4)
    # plt.rcParams["font.size"] = 20
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
    # plt.close()

    corr=np.corrcoef(targ,pred)
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    plt.figure(figsize=(8, 8))
    plt.scatter(targ, pred, alpha=0.6, marker='o', label='Void fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
    plt.close()

    # target_layer = None
    # if hasattr(model, 'conv3'):
    #     target_layer = model.conv3
    # elif hasattr(model, 'layer3'):
    #     target_layer = model.layer3
    # else:
    #     for m in reversed(list(model.modules())):
    #         if isinstance(m, torch.nn.Conv1d):
    #             target_layer = m
    #             break
    # if target_layer is None:
    #     raise RuntimeError("Could not find a suitable layer for Grad-CAM.")

    # gradcam_output_dir = os.path.join(odir, "gradcam")
    # gradcam = GradCAM1d(model, target_layer)
    # grad_cam_map = gradcam(xgradcam)
    # gradcam.remove_hooks()

    # plt.figure(figsize=(10,6))
    # plt.rcParams['font.size'] = 20
    # amp=10
    # ta=np.arange(W)
    # xgradcam=xgradcam.detach().cpu()
    # xgradcamplt=xgradcam.numpy().squeeze(0)
    # plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    # plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    # # plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    # # plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    # # plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    # grad_cam_map *= 30
    # grad_cam_map -= amp
    # plt.legend(loc='upper right')
    # plt.plot(ta,grad_cam_map[:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, grad_cam_map[:],-amp,
    #         color='red',alpha=0.1)
    # plt.xlabel('Time Axis')
    # plt.ylabel('Pressure [-]')
    # plt.ylim(-amp,2*amp)
    # plt.yticks([-amp,0,amp,2*amp],[f"{-amp}","0","0",f"{amp}"])
    # # plt.ylim(-amp,5*amp)
    # # plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    # plt.xlim(0,W)
    # plt.grid(True)
    # # plt.xlim(35,40)
    # plt.tight_layout()
    # new_save_path = os.path.join(odir, f"gradcam{k}.png")
    # plt.savefig(new_save_path)
    # plt.close()

    # model.eval()
    # model.zero_grad()

    # # 1. まずデバイスに送る
    # xgradcam = xgradcam.to(device)

    # # 2. デバイス移動「後」に勾配追跡をONにする
    # xgradcam.requires_grad_()

    # # 3. 予測
    # output = model(xgradcam)

    # # 4. 逆伝播（クラス1に注目する場合）
    # loss = output[0]
    # loss.backward()

    # # 5. 勾配を取り出す（.gradプロパティに格納されている）
    # if xgradcam.grad is not None:
    #     grads = xgradcam.grad.data # ここで [1, 5, 2500] が取得できる
        
    #     # --- チャネル別重要度スコアの計算 ---
    #     # abs()をとって時間方向に平均
    #     salimap = grads.abs().squeeze(0)
        
    #     # 0~1に正規化（安全のために 1e-8 を足す）
    #     salimap = salimap / (salimap.max() + 1e-8)
        
    #     salimap = salimap.cpu().numpy()
    #     salimap = salimap*10
    #     print(salimap.shape)
    # else:
    #     print("勾配が取得できませんでした。requires_gradの位置を確認してください。")


    # plt.figure(figsize=(10,6))
    # plt.rcParams['font.size'] = 20
    # amp=10
    # xgradcam=xgradcam.detach().cpu()
    # xgradcamplt=xgradcam.numpy().squeeze(0)
    # plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    # plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    # # plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    # # plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    # # plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    # plt.legend(loc='upper right')
    # plt.plot(ta,salimap[0,:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, salimap[0,:],0,
    #         color='red',alpha=0.1)
    # plt.plot(ta,amp+salimap[1,:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, amp+salimap[1,:],amp,
    #         color='red',alpha=0.1)
    # # plt.plot(ta,2*amp+salimap[2,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 2*amp+salimap[2,:],2*amp,
    # #         color='red',alpha=0.1)
    # # plt.plot(ta,3*amp+salimap[3,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 3*amp+salimap[3,:],3*amp,
    # #         color='red',alpha=0.1)
    # # plt.plot(ta,4*amp+salimap[4,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 4*amp+salimap[4,:],4*amp,
    # #         color='red',alpha=0.1)
    # plt.xlabel('Time Axis')
    # plt.ylabel('Pressure [-]')
    # plt.ylim(-amp,2*amp)
    # plt.yticks([-amp,0,amp,2*amp],[f"{-amp}","0","0",f"{amp}"])
    # # plt.ylim(-amp,5*amp)
    # # plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    # plt.xlim(0,W)
    # plt.grid(True)
    # # plt.xlim(35,40)
    # plt.tight_layout()
    # new_save_path = os.path.join(odir, f"saliency{k}.png")
    # plt.savefig(new_save_path)
    # plt.close()

res_list=np.array(res_list)
coefs_list=np.array(coefs_list)
reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"res_list:{res_list}")
print(f"coefs_list:{coefs_list}")
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"res:{np.mean(res_list)}")
print(f"coefs:{np.mean(coefs_list)}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")

(315, 2, 2500)
(315,)
torch.Size([315, 2, 2500])
109569


/tmp/ipykernel_2655947/2685038901.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


(48,)
(48,)
coefficient void: 0.8824124055828574
re void: 0.05639940174478198
109569
(48,)
(48,)
coefficient void: 0.8268349212650058
re void: 0.04836358986678468
109569
(48,)
(48,)
coefficient void: 0.7722871357632052
re void: 0.08717179392790182
109569
(48,)
(48,)
coefficient void: 0.8592132125335791
re void: 0.05236340955419495
109569
(48,)
(48,)
coefficient void: 0.7673884432768617
re void: 0.06572777424459388
109569
(48,)
(48,)
coefficient void: 0.8442819833311496
re void: 0.06772545756913823
109569
(48,)
(48,)
coefficient void: 0.8571095467766693
re void: 0.07162785696070464
109569
(48,)
(48,)
coefficient void: 0.8705517515826754
re void: 0.04631580151889192
109569
(48,)
(48,)
coefficient void: 0.8474515395119966
re void: 0.05801332471112415
109569


KeyboardInterrupt: 

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_two_raw_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/t_train_two_raw_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_2raw_std_nn")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=64
W=2500
lr=0.001
N=100

class SimpleNN(nn.Module):
    def __init__(self, input_length):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_length,4096)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4096,2)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x.squeeze(1)

print(x_train.shape)
x_train=x_train.reshape(x_train.shape[0],-1)
# x_train=x_train[:,[1,3,4],:]
print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
# x=x.unsqueeze(1)
print(x.shape)
amp=10
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize
trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
testloader=DataLoader(testset,batch_size=1,shuffle=False)

for k in range(10):
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=B,shuffle=False)
    model=SimpleNN(W*C).to(device)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l1*l1n+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    print(pred.shape)

    corr=np.corrcoef(targ[:,0],pred[:,0])
    coef=corr[0,1]
    print(f'coefficient solid: {coef}')
    # re=np.mean(np.abs(targ[:,0]-pred[:,0])/(targ[:,0]+1e-7))
    re=np.sqrt(np.sum(np.power(targ[:,0]-pred[:,0],2))/targ.shape[0])
    print(f're solid: {re}')
    res_list.append(re)
    coefs_list.append(coef)
    corr=np.corrcoef(targ[:,1],pred[:,1])
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ[:,1]-pred[:,1],2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,0], pred[:,0], alpha=0.6, marker='o', label='Solid fraction')
    plt.plot([-1, 1], [-1, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(-0.0, 0.05)
    plt.ylim(-0.0, 0.05)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,1], pred[:,1], alpha=0.6, marker='o', label='Void fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
    plt.close()

res_list=np.array(res_list)
coefs_list=np.array(coefs_list)
reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"res_list:{res_list}")
print(f"coefs_list:{coefs_list}")
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"res:{np.mean(res_list)}")
print(f"coefs:{np.mean(coefs_list)}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")


(315, 2, 2500)
(315, 5000)
torch.Size([315, 5000])


NameError: name 'l1' is not defined